In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('MachineLearning') \
    .master('local[*]') \
    .getOrCreate()

In [2]:
input_path = '/home/jovyan/work/data/'
users_path = f'{input_path}cleaned_data_users/'

In [397]:
df = spark.read.csv(users_path, multiLine=True, header=True, inferSchema=True)

In [398]:
print(f'Dataframe rows number: {df.count()}')

Dataframe rows number: 147577


In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [399]:
cols = ['_AccountId', 'forum', 'ActivityDays', 
        'comments_number', 'edits_of_answers', 'edits_of_question', 'created_questions', 
        'comments_every_day', 'activity_answer_edits', 'activity_question_edits', 'activity_questions']

In [400]:
df = df[cols]

In [401]:
df.printSchema()

root
 |-- _AccountId: integer (nullable = true)
 |-- forum: string (nullable = true)
 |-- ActivityDays: double (nullable = true)
 |-- comments_number: double (nullable = true)
 |-- edits_of_answers: double (nullable = true)
 |-- edits_of_question: double (nullable = true)
 |-- created_questions: double (nullable = true)
 |-- comments_every_day: double (nullable = true)
 |-- activity_answer_edits: double (nullable = true)
 |-- activity_question_edits: double (nullable = true)
 |-- activity_questions: double (nullable = true)



'OneHotEncoder'

In [402]:
str_cols = ['forum']
encoder = OneHotEncoder(inputCol='forum', outputCol='forum_features')

In [409]:
# unique list of forums
forums_list = df.select('forum').distinct().rdd.map(lambda r: r[0]).collect()
#forums_list

['chemistry',
 'mythology',
 'spanish',
 'french',
 'windowsphone',
 'italian',
 'portuguese',
 'vegetarianism']

In [404]:
forum_udf = udf(lambda x: 1 if x == forum else 0)

In [405]:
for forum in forums_list:
    forum_udf = udf(lambda x: 1 if x == forum else 0)
    df = df.withColumn(f'forum_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_days_{forum}', forum_udf('forum'))
    df = df.withColumn(f'comments_number_{forum}', forum_udf('forum'))
    df = df.withColumn(f'answer_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'question_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'created_questions_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_comments_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_answer_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_question_edits_{forum}', forum_udf('forum'))
    df = df.withColumn(f'activity_questions_{forum}', forum_udf('forum'))

In [406]:
exprs = {x: 'sum' for x in df.columns if x not in cols}

In [407]:
ddf=df.groupBy('_AccountId').agg(exprs)

In [408]:
# number of columns
len(ddf.columns)

81

Features

In [133]:
from pyspark.ml.feature import VectorAssembler

In [410]:
features = [col for col in ddf.columns if col != '_AccountId']

In [411]:
vector_assembler = VectorAssembler(inputCols = features, outputCol= "features_output")
data_training_and_test = vector_assembler.transform(ddf)

Normalization

In [412]:
from pyspark.ml.feature import Normalizer

In [413]:
normalizer = Normalizer(p=1, inputCol="features_output", outputCol="features_norm")

In [414]:
data_training_and_test = normalizer.transform(data_training_and_test)

#### Train and Test Data

In [432]:
(training_data, test_data) = data_training_and_test.randomSplit([0.7, 0.3])

### Modeling Fitting and Tunning
#### Grid Search - Finding the best parameters

In [415]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import *
from pyspark.ml.feature import PCA

In [416]:
very_small_sample = data_training_and_test.sample(False, 0.001).cache()
very_small_sample.count()

140

Dimension Reduction/feature selection

In [417]:
pca_model = PCA(inputCol = "features_norm", outputCol = "pca_features_cv_norm")
km = KMeans(seed=1, featuresCol="pca_features_cv_norm")
ppl_cv = Pipeline(stages = [pca_model, km])

In [418]:
ddf_length = len(ddf.columns)
from builtins import round as base_round
pca_nums = [base_round(i*ddf_length, 0) for i in [0.3, 0.5, 0.75]]
pca_nums

[24.0, 40.0, 61.0]

In [427]:
paramGrid = ParamGridBuilder() \
      .addGrid(pca_model.k, pca_nums) \
      .addGrid(km.k, [5, 8, 10]) \
      .build()

In [429]:
crossval = CrossValidator(estimator = ppl_cv, 
                          estimatorParamMaps=paramGrid,
                          evaluator = ClusteringEvaluator(featuresCol='pca_features_cv_norm', predictionCol='prediction'),
                          numFolds=4)

In [430]:
cv_model = crossval.fit(very_small_sample)

In [433]:
cv_predictions = cv_model.transform(test_data)
evaluator = ClusteringEvaluator(featuresCol='pca_features_cv_norm', predictionCol='prediction')
accuracy = evaluator.evaluate(cv_predictions)
accuracy

0.9338317823288612

In [434]:
cv_model.avgMetrics

[0.680244805460872,
 0.8825390597751505,
 0.8789360711909354,
 0.7235465042690128,
 0.8825390597751505,
 0.8721687266933276,
 0.7006112809685898,
 0.8838346891214681,
 0.8789360711909354]

##### Best model parameters

In [440]:
best_model = cv_model.bestModel
print(f'PCA parameters:\n'
      f'{best_model.stages[0]}')
print(f'KMeans parameters:\n'
      f'{best_model.stages[1]}')

PCA parameters:
PCAModel: uid=PCA_d568cc7898b3, k=61
KMeans parameters:
KMeansModel: uid=KMeans_8f47fa9e8b2f, k=8, distanceMeasure=euclidean, numFeatures=61


In [444]:
kmeans_num = best_model.stages[1].getK()
pca_num = best_model.stages[0].getK()

Modeling fitting and tuning

In [448]:
pca_model = PCA(k=pca_num, inputCol = "features_norm", outputCol = "pca_features_norm")
tuned_model = KMeans(k=kmeans_num, seed=1, featuresCol="pca_features_norm")
tuned_ppl = Pipeline(stages = [pca_model, tuned_model])

In [445]:
#kmeans_model = KMeans(k=kmeans_num, seed=1, featuresCol='features_norm')
#kmeans_model_pca = KMeans(k=kmeans_num, seed=1, featuresCol='pca_features_cv_norm')

In [449]:
tuned_ppl = tuned_ppl.fit(training_data)

In [450]:
# kmeans_model = kmeans_model.fit(training_data)
# kmeans_model_pca = kmeans_model_pca.fit(training_data)

In [452]:
centers = tuned_ppl.stages[1].clusterCenters()
len(centers)

8

In [457]:
train_predictions = tuned_ppl.transform(training_data)
evaluator = ClusteringEvaluator(featuresCol='pca_features_norm', predictionCol='prediction')
train_accuracy = evaluator.evaluate(train_predictions)
train_accuracy

0.9318446689509495

#### Evaluation
Evaluator for Clustering results, which expects two input columns: prediction and features. The metric computes the Silhouette measure using the squared Euclidean distance.

The Silhouette is a measure for the validation of the consistency within clusters. It ranges between 1 and -1, where a value close to 1 means that the points in a cluster are close to the other points in the same cluster and far from the points of the other clusters.

In [455]:
predictions = tuned_ppl.transform(test_data)
evaluator = ClusteringEvaluator(featuresCol='pca_features_norm', predictionCol='prediction')
test_accuracy = evaluator.evaluate(predictions)
test_accuracy

0.9337305689924346

### Save and Load Model

In [458]:
tuned_ppl.write().overwrite().save(input_path+'fitted_model')

In [459]:
from pyspark.ml import PipelineModel
mm=PipelineModel.load(input_path+'fitted_model')

In [460]:
mm.stages

[PCAModel: uid=PCA_a97d23f3c9a0, k=61,
 KMeansModel: uid=KMeans_7cb123786c9a, k=8, distanceMeasure=euclidean, numFeatures=61]

In [461]:
mm.stages[1].clusterCenters()

[array([-9.07012683e-02,  2.55529383e-01,  2.95072427e-02,  1.09302677e-01,
        -9.21471670e-03,  3.91142966e-03,  3.25584348e-02, -1.11094183e-01,
         6.15370669e-03, -1.03679202e-02, -3.48046586e-04, -1.41531288e-03,
        -2.01440638e-03,  1.26039061e-03,  2.33022097e-04, -8.54520823e-06,
        -5.01175288e-05,  2.55087683e-04,  7.83422287e-05, -2.84044696e-04,
         9.66311303e-05,  2.20194906e-04,  6.20143780e-06, -2.51816225e-05,
         1.56907972e-04, -4.92673101e-05,  3.17959579e-05,  1.02894957e-04,
         1.79970124e-04,  1.09123025e-04, -2.96507771e-05, -2.75900186e-05,
         8.00492948e-06,  4.33072390e-05,  2.69418072e-05,  3.29245950e-05,
        -1.60578942e-05,  9.48472303e-06, -5.42159169e-05,  4.18754474e-05,
        -4.17801831e-06, -3.51767559e-06,  3.37502167e-05,  3.37412280e-05,
         1.43795559e-05,  1.10334850e-05,  1.81169559e-06, -1.14286023e-06,
        -3.21371529e-07,  7.30590143e-07, -1.07605576e-06,  6.10333094e-09,
        -4.6